In [4]:
# coding: utf-8

'''
This module is formatting  user config with facts and entities,
creating entity index and producing all variants for entity-match
'''

import pymorphy2
import gensim

import re
import os
from tqdm import tqdm
from operator import itemgetter
from scipy import stats
import random
from functools import lru_cache
import pandas as pd
import dill as pickle

In [2]:
def open_model(m):
    if m.endswith('.vec'):
        model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False)
    elif m.endswith('.bin'):
        model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
    else:
        model = gensim.models.Word2Vec.load(m)
    return model

In [3]:
# common words
f = open(r'/media/mi_air/0F0B7DDE62EEA81E/vector/intersec_vocab','r', encoding='utf8').read().split('\n')
f

['взрывать_VERB',
 'скандалистка_NOUN',
 'волнистый_ADJ',
 'рескрипт_NOUN',
 'зипун_NOUN',
 'расхристанный_ADJ',
 'миниатюра_NOUN',
 'серость_NOUN',
 'радиоцентр_NOUN',
 'вареник_NOUN',
 'удовлетворенный_ADJ',
 'поилка_NOUN',
 'самоотвержение_NOUN',
 'соседка_NOUN',
 'двухтысячелетний_ADJ',
 'зреть_VERB',
 'нью-йорк_PROPN',
 'умыкание_NOUN',
 'жердина_NOUN',
 'опустошенность_NOUN',
 'показывать_VERB',
 'повенчать_VERB',
 'прыжковый_ADJ',
 'гамсун_PROPN',
 'мвд::рф_PROPN',
 'виртуоз_NOUN',
 'учение_NOUN',
 'накрывать_VERB',
 'живьем_ADV',
 'засушивать_VERB',
 'меньшой_ADJ',
 'классицистический_ADJ',
 'кораблев_PROPN',
 'имяславец_NOUN',
 'льдистый_ADJ',
 'стерх_NOUN',
 'долбленка_NOUN',
 'выскребать_VERB',
 'дубленый_ADJ',
 'страховочный_ADJ',
 'шпинель_NOUN',
 'машинистка_NOUN',
 'пасторский_ADJ',
 'вплывать_VERB',
 'синагогальный_ADJ',
 'эберт_PROPN',
 'схватываться_VERB',
 'леонид::гайдай_PROPN',
 'кассия_PROPN',
 'кордон_NOUN',
 'физиотерапия_NOUN',
 'юрий::лужков_PROPN',
 'дидактич

In [5]:
dic_all_words = { 'aranea':{},'taiga':{}, 'rnc':{}}

In [6]:
#aranea
WDIR = r'/media/mi_air/0F0B7DDE62EEA81E/vector'
mname = r'araneum_upos_skipgram_300_2_2018.vec'
mpath = os.path.join(WDIR, mname)
model = open_model(mpath)

In [7]:
num = 100

In [8]:
for w in tqdm(f):
    if w not in dic_all_words['aranea']:
        dic_all_words['aranea'][w] = model.most_similar(w,topn=num)

  0%|          | 0/63290 [00:00<?, ?it/s]/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
100%|██████████| 63290/63290 [29:45<00:00, 35.46it/s]


In [9]:
#rnc
mpath = os.path.join(WDIR, 'ruscorpora_upos_skipgram_300_5_2018.vec')
model = open_model(mpath)

In [10]:
for w in tqdm(f):
    if w not in dic_all_words['rnc']:
        dic_all_words['rnc'][w] = model.most_similar(w,topn=num)

  0%|          | 0/63290 [00:00<?, ?it/s]/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
100%|██████████| 63290/63290 [30:47<00:00, 34.25it/s]


In [11]:
#taiga
mpath = os.path.join(WDIR, 'tayga_1_2.vec')
model = open_model(mpath)

In [12]:
for w in tqdm(f):
    if w not in dic_all_words['taiga']:
        dic_all_words['taiga'][w] = model.most_similar(w,topn=num)

  0%|          | 0/63290 [00:00<?, ?it/s]/home/mi_air/.local/lib/python3.5/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):
100%|██████████| 63290/63290 [42:09<00:00, 25.02it/s]


In [13]:
# теперь найдем пересечение

def get_words(dic):
    return [i[0] for i in dic]

def count_set(list1, list2, list3):
    set1, set2, set3 = set(list1), set(list2), set(list3)
    d = set.intersection(set1, set2, set3)
    u = set(list(set1)+list(set2)+list(set3))
    return len(d)/len(u)



rows_list = []
for word in tqdm(f):
    dict1 = {}
    dict1['word'] = word
    dict1['svodesh/random'] = 'all'
    dict1['% intersection'] = count_set(get_words(dic_all_words['taiga'][word]),get_words(dic_all_words['aranea'][word]),get_words(dic_all_words['rnc'][word]))
    rows_list.append(dict1)

100%|██████████| 63290/63290 [00:20<00:00, 3143.56it/s]


In [14]:
df100 = pd.DataFrame(rows_list) 
df100.head()

,% intersection,svodesh/random,word
0,0.071429,all,взрывать_VERB
1,0.000000,all,скандалистка_NOUN
2,0.011029,all,волнистый_ADJ
3,0.040000,all,рескрипт_NOUN
4,0.072072,all,зипун_NOUN


In [8]:
df100.to_csv(path_or_buf='/media/mi_air/0F0B7DDE62EEA81E/vector/all_word_intersection_100.csv', sep='\t')

In [10]:
df100.head(100)

,Unnamed: 0,% intersection,svodesh/random,word
28320,28320,0.305882,all,неприязнь_NOUN
61672,61672,0.283237,all,недоверие_NOUN
54609,54609,0.278107,all,находчивость_NOUN
48543,48543,0.269006,all,ирландец_NOUN
51682,51682,0.267045,all,радушие_NOUN
20446,20446,0.262570,all,аккуратность_NOUN
45570,45570,0.255814,all,итальянец_NOUN
22865,22865,0.254237,all,идеализм_NOUN
5185,5185,0.252809,all,изобретательность_NOUN
2763,2763,0.252809,all,восхищение_NOUN


In [7]:
df100 = df100.sort_values(['% intersection'], ascending=False)

In [ ]:
pickle.dump(dic_all_words, open('/media/mi_air/0F0B7DDE62EEA81E/vector/all_word_intersection_100.pkl', 'wb'))

In [18]:
model=[]